In [1]:
from collections import Counter, defaultdict
from math import log
import jieba

def readfile(file_path):
    with open(file_path.encode('utf-8')) as f:
        lines = f.readlines()
    return [line.strip() for line in lines]

In [2]:
#先验概率
pos_count = float(len(readfile('pos_train.txt')))
neg_count = float(len(readfile('neg_train.txt')))
sums = pos_count + neg_count
prior = [(neg_count / sums), (pos_count / sums)]
prior

[0.5288782233791589, 0.47112177662084115]

In [3]:
#分词计数
def model(train_file):
    lm = Counter()
    for line in readfile(train_file):
        for word in jieba.cut(line):
            lm[word] += 1
    return lm

#获取所有词
def list():
    lm_pos = model('pos_train.txt')
    lm_neg = model('neg_train.txt')
    s = [lm_neg, lm_pos]
    wordlist = set()
    for i in(0,1):
        for word in s[i].items():
            wordlist.add(word)
    return wordlist
wordlist = list()

#条件概率
def get(train_file):
    v_count = len(wordlist)
    goal = model(train_file)
    sum_ = float(sum(goal.values()))
    get = Counter()
    for word, cnt in goal.items():
        get[word] = (cnt + 1) / (sum_ + v_count + 1)
    get = defaultdict(lambda: 1 / (sum_ + v_count + 1), get)
    return get

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.312 seconds.
Prefix dict has been built succesfully.


In [4]:
#训练词料条件概率
pos_train_lm = get('pos_train.txt')
neg_train_lm = get('neg_train.txt')

In [5]:
#算出测试语料后验概率
def classify(content, pos_train_lm, neg_train_lm, i):
    words = jieba.cut(content)
    result = log(prior[i])
    for word in words:
        #if word in wordlist:
        result += log(pos_train_lm[word] / neg_train_lm[word])
    return result

In [7]:
#计算情感分类
s_test = ['neg_test.txt', 'pos_test.txt']
r = [[0,0],[0,0]]
for i in (0,1):
    for line in open(s_test[i]):
        if classify(line,pos_train_lm, neg_train_lm,i)<0:
            r[i][0] +=1
        else:
            r[i][1] +=1
total = r[0][0]+r[0][1]+r[1][0]+r[1][1]

In [9]:
r

[[4990, 583], [1267, 3698]]

In [8]:
confusion_matrix = ('\n'.join(['{:^15}' * 3] * 3)).format(
    'n='+str(total), 'predicted:neg', 'predicted:pos',
    'actual:neg', r[0][0], r[1][0],
    'actual:pos', r[0][1], r[1][1])
print(confusion_matrix)

    n=10538     predicted:neg  predicted:pos 
  actual:neg        4990           1267      
  actual:pos         583           3698      


In [10]:
4990.0/(4990+1267)

0.7975067923925204

In [11]:
3698.0/(583+3698)

0.863816865218407